In [6]:
import requests
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import urllib.parse
from csv import writer
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import multiprocessing as mp

In [7]:
stop_words = set(stopwords.words('english')) 
stemmer = WordNetLemmatizer()

In [8]:
# init it_industry word for search
it_industry = pd.read_csv("data/jobs_position_category.csv", header=None, encoding='cp1252')

In [9]:
# init indeed
base_url = "https://www.indeed.com/"
pages = 30 # scape 5 page each page include 10 job = 300job

In [10]:
def append_list_as_row(file_name, item):
    with open(file_name, "a+") as file_object:
        # Move read cursor to the start of file.
        file_object.seek(0)
        # If file is not empty then append '\n'
        data = file_object.read(100)
        if len(data) > 0 :
            file_object.write("\n")
        # Append text at the end of file
        file_object.write(item)

In [11]:
def request_query(page):
    response = requests.get(page)
    soup = BeautifulSoup(response.text, "html.parser")
    return soup

In [12]:
def request_query_soup(keyword, page):
    query_search = f"{base_url}jobs?q={urllib.parse.quote(keyword)}&start={page}"
    return request_query(query_search)

In [8]:
# def clean_data(document):
#     document = re.sub(r'\W', ' ', str(document))
    
#     # remove all single characters
#     document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
#     # remove single characters from the start
#     document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
#     # substituting multiple spaces with single space
#     document = re.sub(r'\s+', ' ', document, flags=re.I)
    
#     # remove hashtag #
#     document = re.sub(r'#','',document) 
    
#     # Converting to Lowercase
#     document = document.lower()
    
#     return document

In [9]:
# def split_word(document):
#     tokens = word_tokenize(clean_data(document))
#     tokens = [i for i in tokens if not i in stop_words] # remove stop word
#     tokens = [word for word in tokens if word.isalpha()] # remove alpha
#     tokens = [i for i in tokens if not i.isnumeric()] # remove number
#     tokens = [i for i in tokens if not ' ' in i] # remove space
#     tokens = [stemmer.lemmatize(i) for i in tokens] # root
    
#     document = ' '.join(tokens)
#     return document

In [12]:
# pbar = tqdm(total=len(it_industry) * pages)
# loop each page
def run_scap(keyword, pages=1):
    link_list = []
    keyword_list = []
    print(keyword)
    for page in range(pages):
        page = page * 10
        soup = request_query_soup(keyword, page)

        query_div = soup.find_all("div", {"class": "jobsearch-SerpJobCard"})
        for div in query_div:
            link_list.append(f"https://www.indeed.com/viewjob?jk={div.attrs['data-jk']}&from=serp&vjs=3")
            keyword_list.append(keyword)
            
    return link_list, keyword_list

In [ ]:
def test1(keyword):
    return keyword.shape

In [15]:
# number of core
print("Number of processors: ", mp.cpu_count())

# init pool
pool = mp.Pool(mp.cpu_count())

Number of processors:  4


In [ ]:
results = [pool.apply(test1, args =(keyword)) for keyword in it_industry[0]]
pool.close()

In [38]:
# link_list, keyword_list = run_scap(it_industry[0][0])

In [41]:
df_split = np.array_split(it_industry[0], mp.cpu_count(), axis=0)
df_split

[0    Mobile Application Developer
 1          Database Administrator
 2                   Web Developer
 3            Help Desk Technician
 Name: 0, dtype: object, 4    Network Administrator
 5         Graphic Designer
 6              IT Security
 7        Software Engineer
 Name: 0, dtype: object, 8                          IT Manager
 9                     Systems Analyst
 10                     Data Scientist
 11    Chief Information Officer (CIO)
 Name: 0, dtype: object, 12          Devops Engineer
 13           Cloud Engineer
 14    Business Intelligence
 Name: 0, dtype: object]

In [47]:
[keyword.tolist() for keyword in df_split]

[['Mobile Application Developer',
  'Database Administrator',
  'Web Developer',
  'Help Desk Technician'],
 ['Network Administrator',
  'Graphic Designer',
  'IT Security',
  'Software Engineer'],
 ['IT Manager',
  'Systems Analyst',
  'Data Scientist',
  'Chief Information Officer (CIO)'],
 ['Devops Engineer', 'Cloud Engineer', 'Business Intelligence']]

In [ ]:
link_list, keyword_list = pool.map(run_scap, [keyword for keyword in it_industry[0]])
pool.close()

In [81]:
# load job_link
job_link = pd.read_csv("indeed-scaper/data/job_url.csv", header=None)
job_data = pd.read_csv("indeed-scaper/data/job_scrap.csv", header=None)

In [85]:
print(job_link.shape)
print(job_data.shape)

(4425, 2)
(4425, 3)


In [83]:
job_data = job_data.drop(columns=[3])

In [89]:
job_data_new = pd.concat([job_link, job_data], axis=1)
job_data_new
job_data_new.columns=['job_type', 'link', 'job_title', 'company', 'desc']
job_data_new.dropna(subset = ['job_title', 'desc'], inplace=True)
job_data_new.to_csv('data/job_data.csv', index=None)

In [280]:
# loof each link
header_list = []
company_list = []
desc_list = []
job_link_temp = job_link[1]

with tqdm(total=len(job_link_temp)) as pbar:
    for link in job_link_temp:
        soup = request(link)
        header = soup.find("h3", {"class": "jobsearch-JobInfoHeader-title"})
        header = header.getText() if header else ""
        company = soup.find("div", {"class": "jobsearch-InlineCompanyRating"})
        company = company.getText() if company else ""
        desc = soup.find("div", {"class": "jobsearch-jobDescriptionText"})
#         desc = split_word(desc.getText()) if desc else ""

        df = pd.DataFrame({'job_title': [header], 'company': [company], 'desc': [desc]})
        df.to_csv('data/job_scrap.csv', mode='a', header=False, index=None)
        pbar.update(1)

100%|██████████| 5340/5340 [2:01:13<00:00,  1.36s/it]  


In [302]:
job_scrap = pd.read_csv("data/job_scrap.csv", header=None)

In [45]:
job_data = pd.concat([job_link, job_scrap], axis=1)
job_data.columns=['job_type', 'link', 'job_title', 'company', 'desc']
job_data.dropna(subset = ['job_title', 'desc'], inplace=True)
job_data.to_csv('data/job_data.csv', index=None)

NameError: name 'job_scrap' is not defined